In [ ]:
%load_ext autoreload
%autoreload 2

In [11]:
import sys
sys.path.append("..")
sys.path.append(".")
import torch
import numpy as np
import copy
import plotly
import plotly.graph_objs as go
import pickle
from datetime import datetime
plotly.offline.init_notebook_mode(connected=True)

import robust_value_approx.samples_generator as samples_generator
import robust_value_approx.samples_buffer as samples_buffer
import robust_value_approx.value_approximation as value_approximation
import robust_value_approx.training_log as training_log
import robust_value_approx.controllers as controllers

import pendulum_utils

In [30]:
vf, sys = pendulum_utils.get_value_function(10)
V = vf.get_value_function()

In [31]:
x0 = torch.tensor([np.pi/2, 0.], dtype=vf.dtype)
v, res = V(x0)

In [32]:
res

{'v': 2.8771905602458485,
 'x_traj': [tensor([1.5708, 0.0000], dtype=torch.float64),
  tensor([1.6987, 1.2792], dtype=torch.float64),
  tensor([2.0402, 2.1359], dtype=torch.float64),
  tensor([2.4290, 1.7518], dtype=torch.float64),
  tensor([2.7206, 1.1641], dtype=torch.float64),
  tensor([2.9081, 0.7111], dtype=torch.float64),
  tensor([3.0217, 0.4249], dtype=torch.float64),
  tensor([3.0895, 0.2530], dtype=torch.float64),
  tensor([3.1266, 0.1181], dtype=torch.float64),
  tensor([3.1390, 0.0063], dtype=torch.float64)],
 'u_traj': [tensor([13.3220], dtype=torch.float64),
  tensor([19.6764], dtype=torch.float64),
  tensor([9.2579], dtype=torch.float64),
  tensor([4.2018], dtype=torch.float64),
  tensor([1.8768], dtype=torch.float64),
  tensor([0.8289], dtype=torch.float64),
  tensor([0.3233], dtype=torch.float64),
  tensor([-0.0179], dtype=torch.float64),
  tensor([-0.4874], dtype=torch.float64),
  tensor([-0.3964], dtype=torch.float64)],
 'alpha_traj': tensor([[1., 1., 1., 1., 1., 1.,

In [16]:
x0_lo = torch.tensor([-1., -1.], dtype=vf.dtype)
x0_up = torch.tensor([1., 1.], dtype=vf.dtype)
gen = samples_generator.RandomSampleGenerator(vf, x0_lo, x0_up)
gen.generate_samples(10, show_progress=True, rollout=True)

Progress: [########################################] 100.0%


(tensor([[ 8.0853e-01,  5.4523e-01,  1.0831e+00,  2.2010e+00,  1.6123e+00,
           3.0909e+00,  2.1679e+00,  2.4653e+00,  2.5747e+00,  1.6021e+00,
           2.8309e+00,  9.6002e-01,  2.9834e+00,  5.6513e-01,  3.0732e+00,
           3.3309e-01,  3.1220e+00,  1.5460e-01],
         [ 1.2818e-01, -6.0844e-01,  3.1763e-01,  2.5029e+00,  9.9195e-01,
           4.2403e+00,  1.7703e+00,  3.5434e+00,  2.3535e+00,  2.2885e+00,
           2.7165e+00,  1.3415e+00,  2.9280e+00,  7.7291e-01,  3.0501e+00,
           4.4826e-01,  3.1155e+00,  2.0619e-01],
         [ 2.6232e-03, -1.4905e-01,  2.7432e-01,  2.8660e+00,  9.9964e-01,
           4.3871e+00,  1.7934e+00,  3.5506e+00,  2.3738e+00,  2.2530e+00,
           2.7298e+00,  1.3071e+00,  2.9353e+00,  7.4843e-01,  3.0534e+00,
           4.3238e-01,  3.1165e+00,  1.9849e-01],
         [ 3.8898e-01,  7.7447e-01,  7.4390e-01,  2.7747e+00,  1.3883e+00,
           3.6698e+00,  2.0415e+00,  2.8621e+00,  2.5098e+00,  1.8208e+00,
           2.7991e+00,  1

In [33]:
x0_lo = torch.tensor([-1., -1.], dtype=vf.dtype)
x0_up = torch.tensor([1., 1.], dtype=vf.dtype)
gen = samples_generator.GridSampleGenerator(vf, x0_lo, x0_up)
x, v, t = gen.generate_samples([10, 10], show_progress=True, rollout=True)

Progress: [########################################] 100.0%


In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x[:,0],
    y=x[:,1],
    mode='markers',
))
fig.show()

In [34]:
buff = samples_buffer.TimedSamplesBuffer(x.shape[1], v.shape[1], vf.dtype)
buff.add_samples(x, v, t)

In [38]:
v[0,:]

tensor([1.2009e+01, 8.1272e+00, 4.1060e+00, 1.6174e+00, 4.9520e-01, 1.3534e-01,
        3.6564e-02, 1.0782e-02, 4.2946e-03, 1.6026e-03], dtype=torch.float64)